# BOOK RECOMENDATION SYSTEM WITH VECTOR STORE EMBEDDING AND LLMs

### 1. Convert csv data to vector strings and save embeddings 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('books.csv')
df

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6805,9788185300535,8185300534,I Am that,Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0
6806,9788185944609,8185944601,Secrets Of The Heart,NaN,Khalil Gibran,Mysticism,http://books.google.com/books/content?id=XcrVp...,NaN,1993.0,4.08,74.0,324.0
6807,9788445074879,8445074873,Fahrenheit 451,NaN,Ray Bradbury,Book burning,NaN,NaN,2004.0,3.98,186.0,5733.0
6808,9789027712059,9027712050,The Berlin Phenomenology,NaN,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,210.0,0.0


In [4]:
def textual_representation(row):
    textual_representation = f"""Title: {row['title']}
Author: {row['authors']}
Category: {row['categories']}
Description: {row['description']}
Publication Year: {row['published_year']}
Average Rating: {row['average_rating']}
Number of Pages: {row['num_pages']}
"""
    return textual_representation

In [6]:
df.iloc[:5].apply(textual_representation, axis=1)

0    Title: Gilead\nAuthor: Marilynne Robinson\nCat...
1    Title: Spider's Web\nAuthor: Charles Osborne;A...
2    Title: The One Tree\nAuthor: Stephen R. Donald...
3    Title: Rage of angels\nAuthor: Sidney Sheldon\...
4    Title: The Four Loves\nAuthor: Clive Staples L...
dtype: object

In [7]:
df.iloc[:5].apply(textual_representation, axis=1).to_list()

['Title: Gilead\nAuthor: Marilynne Robinson\nCategory: Fiction\nDescription: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is

In [8]:
print(df.iloc[:5].apply(textual_representation, axis=1).values[0])

Title: Gilead
Author: Marilynne Robinson
Category: Fiction
Description: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a so

In [9]:
df['textual_representation'] = df.apply(textual_representation, axis=1)

In [10]:
import faiss
import requests
import numpy as np

dim = 4096 # Dimension of the embeddings because thats the dimensionality response we get from olamma2

index = faiss.IndexFlatL2(dim)  # L2 distance index

X = np.zeros((len(df['textual_representation']), dim), dtype=np.float32)

In [12]:
for i, representation in enumerate(df['textual_representation']):
    if i % 100 == 0:
        print(f"Processing row {i} of {len(df['textual_representation'])}")
    
    response = requests.post('http://localhost:11434/api/embeddings', 
                             json={
                                 'model' : 'llama2',
                                 'prompt' : representation
                             })
    embedding = response.json()['embedding']
    X[i] = np.array(embedding, dtype=np.float32)
    
index.add(X)  # Add embeddings to the index

Processing row 0 of 6810


KeyboardInterrupt: 

In [14]:
faiss.write_index(index, 'index')  # Save the index to a file

### 2. Load embeddings from index

In [15]:
index = faiss.read_index('index')  # Load the index from a file

In [16]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000022AF37C29A0> >

In [17]:
favorite_book = df.iloc[4533]
favorite_book

isbn13                                                        9780749307844
isbn10                                                           0749307846
title                               How to Win Friends and Influence People
subtitle                                                                NaN
authors                                                       Dale Carnegie
categories                                                  Conduct of life
thumbnail                 http://books.google.com/books/content?id=aO7CQ...
description               Dale Carnegie aims to show how to makes friend...
published_year                                                       1990.0
average_rating                                                         4.18
num_pages                                                             256.0
ratings_count                                                         199.0
textual_representation    Title: How to Win Friends and Influence People...
Name: 4533, 

In [19]:
response = requests.post('http://localhost:11434/api/embeddings', 
                            json={
                                'model' : 'llama2',
                                'prompt' : favorite_book['textual_representation']
                            })

In [20]:
embedding = np.array([response.json()['embedding']], dtype=np.float32)

In [21]:
D, I = index.search(embedding, 10)  # Search for the 10 nearest neighbors

In [22]:
best_matches = np.array(df['textual_representation'])[I.flatten()]

In [23]:
for match in best_matches:
    print(match)
    print('-' * 80)

Title: 'I'm Telling You Stories'
Author: Helena Grice;Tim Woods
Category: Literary Criticism
Description: This is a jubilant and rewarding collection of Winterson scholarship--a superb group of essays from a host of fine authors.
Publication Year: 1998.0
Average Rating: 3.7
Number of Pages: 136.0

--------------------------------------------------------------------------------
Title: 'I'm Telling You Stories'
Author: Helena Grice;Tim Woods
Category: Literary Criticism
Description: This is a jubilant and rewarding collection of Winterson scholarship--a superb group of essays from a host of fine authors.
Publication Year: 1998.0
Average Rating: 3.7
Number of Pages: 136.0

--------------------------------------------------------------------------------
Title: 'I'm Telling You Stories'
Author: Helena Grice;Tim Woods
Category: Literary Criticism
Description: This is a jubilant and rewarding collection of Winterson scholarship--a superb group of essays from a host of fine authors.
Publicatio